![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F99+-+Cleanup&file=99+-+Cleanup.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/99%20-%20Cleanup/99%20-%20Cleanup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F99%2520-%2520Cleanup%2F99%2520-%2520Cleanup.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/99%20-%20Cleanup/99%20-%20Cleanup.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/99%20-%20Cleanup/99%20-%20Cleanup.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Cleanup All Resources

This notebook is designed to remove all resources by the type created.  By being a general cleanup utility, you can skip sections that you want to remain in use.  This notebook also has a section to delete the GCS bucket created in the `00 - Environment Setup` notebook.

### Resources
- Vertex AI: https://googleapis.dev/python/aiplatform/latest/index.html
- BigQuery: https://googleapis.dev/python/bigquery/latest/index.html
- GCS: https://googleapis.dev/python/storage/latest/client.html

---
## Method 1: Delete The Project
- [Instructions](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)
    - Open Console > IAM & Admin > Settings: [Link](https://console.cloud.google.com/iam-admin/settings)
    - Click Select a project.
    - Select a project you want to delete, and click Open.
    - Click Shut down.
    - Enter the Project ID, then click Shut down.

---
## Method 2: Remove the Items Created By This Notebook

## Setup
When Finished with removing each Vertex AI, GCS, and BigQuery item below, you will then need to stop and delete this notebook instance in Console > Vertex AI > Notebooks.

inputs:

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [ ]:
#PROJECT_ID = "statmike-mlops"
REGION = 'us-central1'

packages:

In [ ]:
from google.cloud import aiplatform
from google.cloud import aiplatform_v1beta1
from google.cloud import bigquery
from google.cloud import storage

parameters:

In [ ]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

clients:

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)
clients['bq'] = bigquery.Client(project=PROJECT_ID)
clients['fs'] = aiplatform_v1beta1.FeaturestoreServiceClient(client_options=client_options)
clients['viz'] = aiplatform_v1beta1.VizierServiceClient(client_options=client_options)
clients['md'] = aiplatform_v1beta1.MetadataServiceClient(client_options=client_options)
clients['gcs'] = storage.Client()

---
## BigQuery
- Delete the dataset `digits`

list of datasets in the project:

In [ ]:
datasets = list(clients['bq'].list_datasets())

In [ ]:
for ds in datasets: print(ds.dataset_id)

delete each dataset:

In [ ]:
for ds in datasets:
    print(ds.dataset_id)
    clients['bq'].delete_dataset(ds.dataset_id,delete_contents=True,not_found_ok=True)

---
## Vertex AI > Datasets
- Delete datasets

In [ ]:
datasets = aiplatform.TabularDataset.list()

In [ ]:
for ds in datasets:
    print(ds.resource_name)
    aiplatform.TabularDataset(dataset_name=ds.resource_name).delete()

In [ ]:
datasets = aiplatform.TimeSeriesDataset.list()

In [ ]:
for ds in datasets:
    print(ds.resource_name)
    aiplatform.TimeSeriesDataset(dataset_name=ds.resource_name).delete()

---
## Vertex AI > Features
- delete feature store including entity types and features

In [ ]:
fs = clients['fs'].list_featurestores(parent=PARENT)

In [ ]:
for f in fs:
    print(f.name)
    clients['fs'].delete_featurestore(request = aiplatform_v1beta1.types.DeleteFeaturestoreRequest(name=f.name,force=True))

---
## Vertex AI > Pipelines
* Delete Managed Pipelines

In [ ]:
pl = aiplatform.PipelineJob.list()

In [ ]:
for p in pl:
    print(p.name)
    p.delete()

---
## Vertex AI > Training
- delete various jobs types
- some create with aiplatform.* and other created by aiplatform.gapic.*

### Vertex AI > Training > Training Pipelines
- Delete training pipelines

In [ ]:
jobs = aiplatform.CustomTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

In [ ]:
jobs = aiplatform.CustomPythonPackageTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

In [ ]:
jobs = aiplatform.AutoMLTabularTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

In [ ]:
jobs = aiplatform.AutoMLForecastingTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

### Vertex AI > Training > Custom Job
- Delete custom training jobs

In [ ]:
jobs = aiplatform.CustomJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

### Vertex AI > Training > Hyperparameter Tuning Job

In [ ]:
jobs = aiplatform.HyperparameterTuningJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

---
## Vertex AI > Experiments > Experiments
Needs to run before the next section or it may fail due to missing the backing Tensorboards

In [ ]:
exps = aiplatform.Experiment.list()

In [ ]:
for exp in exps:
    runs = aiplatform.ExperimentRun.list(experiment = exp.name)
    print(f'Experiment: {exp.name}')
    for run in runs:
        print(f'Run: {run.name}')
        run.delete(delete_backing_tensorboard_run = False)
    exp.delete(delete_backing_tensorboard_runs = False)

## Vertex AI > Experiments > Tensorboard Instances

In [ ]:
tbs = aiplatform.Tensorboard.list()

In [ ]:
for tb in tbs:
    print(tb.resource_name)
    exps = aiplatform.TensorboardExperiment.list(tensorboard_name = tb.resource_name)
    for exp in exps:
        print(exp.name)
        exp.delete()
    tb.delete()

## Vertex AI > Experiments > Studies
Vizier Studies

In [ ]:
studies = clients['viz'].list_studies(parent=PARENT)

In [ ]:
for study in studies:
    print(study.name)
    clients['viz'].delete_study(name=study.name)

---
## Vertex AI > Endpoints
- Delete endpoints forcing deployed models to undeploy

In [ ]:
endpoints = aiplatform.Endpoint.list()

In [ ]:
for ep in endpoints:
    print(ep.resource_name)
    ep.delete(force=True)

---
## Vertex AI > Models
- Delete endpoints first (contain deployed models)
- Delete uploaded models

In [ ]:
models = aiplatform.Model.list()

In [ ]:
for model in models:
    print(model.resource_name)
    model.delete()

---
## Vertex AI > Batch Predictions
- delete all batch prediction jobs

In [ ]:
bps = aiplatform.BatchPredictionJob.list()

In [ ]:
for bp in bps:
    print(bp.resource_name)
    bp.delete()

---
## Vertex AI > Metadata

In [ ]:
md = clients['md'].list_metadata_stores(parent=PARENT)

In [ ]:
for m in md:
    print(m.name)
    arts = clients['md'].list_artifacts(parent=m.name)
    for a in arts:
        print(a.name)
        clients['md'].delete_artifact(name=a.name)
    if m.name.split('/')[-1] != 'default':
        print('delete would happen here - update code to make this happen')
        #clients['md'].get_metadata_store(name=m.name)#.purge_artifacts(parent=m.name)

---
## Storage Bucket Removal

- delete contents and storage bucket `statmike-mlops` (named after project)

In [ ]:
bucket = clients['gcs'].get_bucket(PROJECT_ID)

In [ ]:
#bucket.delete(force=True)